In [1]:
from cbench.commands import *
from cbench import state
from plumbum.machines.paramiko_machine import ParamikoMachine
from time import sleep
import _thread

In [2]:
state.RUN_NAME = "Baseline_SCAN_6_vs_m4.2xlarge_100_2"
state.WORKLOAD = "workloads/workload_scan"

In [ ]:
print("Instances:")
list_instances()
print("Loading state from AWS..")
load_state()
print("Cluster instances: {0}".format(state.CLUSTER_INSTANCES))

In [3]:
create_instances(6, state.CLUSTER_INSTANCES, type="i2.xlarge")
create_instances(1, state.YCSB_INSTANCES, 'ycsb', type="m4.2xlarge")

2016-01-17 12:47:22,104 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'create_instances' args: (6, []) kwargs: {'type': 'i2.xlarge'}
2016-01-17 12:47:23,642 INFO Created instance with id: i-611194dd private ip: 172.31.31.240 public ip: None
2016-01-17 12:47:24,250 INFO Created instance with id: i-db119467 private ip: 172.31.20.160 public ip: None
2016-01-17 12:47:24,876 INFO Created instance with id: i-d8119464 private ip: 172.31.16.199 public ip: None
2016-01-17 12:47:25,503 INFO Created instance with id: i-c110957d private ip: 172.31.28.40 public ip: None
2016-01-17 12:47:26,121 INFO Created instance with id: i-011095bd private ip: 172.31.20.88 public ip: None
2016-01-17 12:47:26,710 INFO Created instance with id: i-d9119465 private ip: 172.31.29.92 public ip: None
2016-01-17 12:47:26,711 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 end of 'create_instances'
2016-01-17 12:47:26,712 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'create_instances' args: (1, [], 'ycsb') 

In [ ]:
print(state.CLUSTER_INSTANCES)

In [4]:
#Wait for Instances to finish boot
sleep(5 * 60)

In [5]:
# create the cluster
create_cluster(state.CLUSTER_INSTANCES)

2016-01-17 12:52:30,365 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'create_cluster' args: (['i-611194dd', 'i-db119467', 'i-d8119464', 'i-c110957d', 'i-011095bd', 'i-d9119465'],) kwargs: {}
2016-01-17 12:52:30,800 INFO Checking instance 'i-611194dd' state: {'Code': 16, 'Name': 'running'}
2016-01-17 12:52:30,800 INFO Connecting to 52.29.60.93
2016-01-17 12:52:30,919 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 12:52:31,172 INFO Authentication (publickey) successful!
2016-01-17 12:52:32,049 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 12:52:32,529 INFO Going to wait for 5 seconds for startup or bootstrap to finish
2016-01-17 12:52:37,532 INFO Command returned:
2016-01-17 12:52:37,534 INFO d4b14f9dc3e67452a6a3dbf9d4c9849993927f56d8ac19ce02b21f87910077a2

2016-01-17 12:52:37,654 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 12:52:37,939 INFO Authentication (publickey) successful!
2016-01-17 12:52:38,870 INFO [chan 1] Open

In [6]:
prepare_benchmark(workload="workloads/workload_scan", name=state.RUN_NAME)

2016-01-17 12:57:57,534 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'prepare_benchmark' args: () kwargs: {'name': 'Baseline_SCAN_6_vs_m4.2xlarge_100_2', 'workload': 'workloads/workload_scan'}
2016-01-17 12:57:57,534 INFO Creating keyspace YCSB and table..
2016-01-17 12:57:57,717 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 12:57:58,000 INFO Authentication (publickey) successful!
2016-01-17 12:57:58,875 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 12:58:03,498 INFO Result: 
2016-01-17 12:58:03,553 INFO Loading YCSB data for workload 'workloads/workload_scan'
2016-01-17 12:58:03,750 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 12:58:04,049 INFO Authentication (publickey) successful!
2016-01-17 13:17:37,431 INFO Result: YCSB Client 0.1
Command line: -db com.yahoo.ycsb.db.CassandraCQLClient -P workloads/workload_base -P workloads/workload_scan -p hosts=172.31.31.240,172.31.20.160,172.31.16.199,172.31.28.40,172.31.20.88 -

In [ ]:
temp = state.CLUSTER_INSTANCES[-1:]

state.CLUSTER_INSTANCES = state.CLUSTER_INSTANCES[:-1]

In [ ]:
state.CLUSTER_INSTANCES.append(temp[0])

In [ ]:
print(state.CLUSTER_INSTANCES)

In [7]:
# Baseline benchmark
bench = _thread.start_new_thread(start_benchmark, (100, []))
sleep(5 * 60)
wait_for_finish()
gather_results()

2016-01-17 13:17:37,609 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'start_benchmark' args: (100, []) kwargs: {}
2016-01-17 13:17:37,610 INFO Starting run 'Baseline_SCAN_6_vs_m4.2xlarge_100_2' with workload 'workloads/workload_scan'
2016-01-17 13:17:37,915 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:17:38,190 INFO Authentication (publickey) successful!
2016-01-17 13:22:37,610 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'wait_for_finish' args: () kwargs: {}
2016-01-17 13:22:37,946 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:22:38,304 INFO Authentication (publickey) successful!
2016-01-17 13:22:39,691 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 13:22:50,332 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:22:50,661 INFO Authentication (publickey) successful!
2016-01-17 13:22:52,063 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 13:23:02,826 INFO Connected (versio

In [8]:
print(state.CLUSTER_INSTANCES)
# Create additional instance
create_instances(1, state.CLUSTER_INSTANCES, type="i2.xlarge")
print("New instance: {0}".format(state.CLUSTER_INSTANCES[-1:]))

['i-611194dd', 'i-db119467', 'i-d8119464', 'i-c110957d', 'i-011095bd', 'i-d9119465']
2016-01-17 13:38:10,356 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 start of 'create_instances' args: (1, ['i-611194dd', 'i-db119467', 'i-d8119464', 'i-c110957d', 'i-011095bd', 'i-d9119465']) kwargs: {'type': 'i2.xlarge'}
2016-01-17 13:38:11,037 INFO Created instance with id: i-5c1297e0 private ip: 172.31.28.6 public ip: None
2016-01-17 13:38:11,038 INFO Run Baseline_SCAN_6_vs_m4.2xlarge_100_2 end of 'create_instances'
New instance: ['i-5c1297e0']


In [ ]:
scale_cluster(state.CLUSTER_INSTANCES[-1:])

In [ ]:
sleep(5)
state.CLUSTER_INSTANCES.append(temp[0])

In [9]:
# Scaling benchmark
state.RUN_NAME = "Scale_SCAN_6_vs_m4.2xlarge_100"
bench = _thread.start_new_thread(start_benchmark, (100, []))
sleep(3 * 60)
scale_cluster(state.CLUSTER_INSTANCES[-1:])
wait_for_finish()
gather_results()

2016-01-17 13:38:11,043 INFO Run Scale_SCAN_6_vs_m4.2xlarge_100 start of 'start_benchmark' args: (100, []) kwargs: {}
2016-01-17 13:38:11,044 INFO Starting run 'Scale_SCAN_6_vs_m4.2xlarge_100' with workload 'workloads/workload_scan'
2016-01-17 13:38:11,213 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:38:11,447 INFO Authentication (publickey) successful!
2016-01-17 13:41:11,043 INFO Run Scale_SCAN_6_vs_m4.2xlarge_100 start of 'scale_cluster' args: (['i-5c1297e0'],) kwargs: {}
2016-01-17 13:41:11,218 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:41:11,481 INFO Authentication (publickey) successful!
2016-01-17 13:41:12,405 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 13:41:15,478 INFO docker exec ['nodetool', 'status'] on i-611194dd returned:
2016-01-17 13:41:15,480 INFO Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens       Owns    Host ID                    

In [ ]:
print(state.SEED_IP)

In [10]:
#Partition benchmark
state.RUN_NAME = "Reduce_SCAN_6_vs_m4.2xlarge_100"
bench = _thread.start_new_thread(start_benchmark, (100, []))
sleep(3 * 60)
remove_cassandra_instance(state.CLUSTER_INSTANCES[-1])
wait_for_finish()
gather_results()

2016-01-17 13:58:44,198 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'start_benchmark' args: (100, []) kwargs: {}
2016-01-17 13:58:44,198 INFO Starting run 'Reduce_SCAN_6_vs_m4.2xlarge_100' with workload 'workloads/workload_scan'
2016-01-17 13:58:44,367 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 13:58:44,628 INFO Authentication (publickey) successful!
2016-01-17 14:01:44,199 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'remove_cassandra_instance' args: ('i-5c1297e0',) kwargs: {}
2016-01-17 14:01:44,370 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-01-17 14:01:44,630 INFO Authentication (publickey) successful!
2016-01-17 14:01:45,527 INFO [chan 1] Opened sftp connection (server version 3)
2016-01-17 14:01:48,123 INFO docker exec ['nodetool', 'status'] on i-611194dd returned:
2016-01-17 14:01:48,123 INFO Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens       Owns    Host ID       

In [ ]:
#state.RUN_NAME = "Reduce_READ_MODIFY_WRITE_4_vs_m4.2xlarge_100_2"
wait_for_finish()
#gather_results()

In [ ]:
gather_results()

In [ ]:
cleanup_logs()

In [11]:
terminate_all()

2016-01-17 15:28:29,990 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'terminate_instance' args: ('i-611194dd',) kwargs: {}
2016-01-17 15:28:29,991 WARNING Going to terminate instance 'i-611194dd'
2016-01-17 15:28:30,390 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 end of 'terminate_instance'
2016-01-17 15:28:30,391 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'terminate_instance' args: ('i-db119467',) kwargs: {}
2016-01-17 15:28:30,392 WARNING Going to terminate instance 'i-db119467'
2016-01-17 15:28:30,672 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 end of 'terminate_instance'
2016-01-17 15:28:30,675 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'terminate_instance' args: ('i-d8119464',) kwargs: {}
2016-01-17 15:28:30,678 WARNING Going to terminate instance 'i-d8119464'
2016-01-17 15:28:30,948 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 end of 'terminate_instance'
2016-01-17 15:28:30,951 INFO Run Reduce_SCAN_6_vs_m4.2xlarge_100 start of 'terminate_instance' args: ('i-c110957d',) k

In [ ]:
from cbench import util

for inst in state.CLUSTER_INSTANCES:
    util.docker_status(inst)